### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# cell for work
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
# cell for work
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0102926,9,1590148016
2,2,0208092,5,1586466072
3,2,0358273,9,1579057827
4,2,10039344,5,1578603053


In [12]:
# more work
reviews.rating.isnull().sum()

0

In [7]:
# and still more
reviews.shape[0]

874426

In [9]:
# and even more - open additional cells as necessary
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

# we end up needing this later
genres = set(genres)
print("The number of genres is {}.".format(len(genres)))

The number of genres is 28.


In [15]:
reviews.rating.max()

10

In [16]:
# Use your findings to match each variable to the correct statement in the dictionary
a = 53968
b = 10
c = 7
d = 31245
e = 15
f = 0
g = 4
h = 712337
i = 28

dict_sol1 = {
'The number of movies in the dataset': d,
'The number of ratings in the dataset': h,
'The number of different genres': i,
'The number of unique users in the dataset': a,
'The number missing ratings in the reviews dataset': f,
'The average rating given across all ratings': c,
'The minimum rating given across all ratings': f,
'The maximum rating given across all ratings': b
}

# Check your solution
t.q1_check(dict_sol1)

That looks good to me!


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp
* Create month/day/year columns from timestamp as dummies

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [20]:
# cell for work
create_date = lambda x: x[-5:-1] if x[-1] == ')' else np.nan
movies['Date'] = movies['movie'].apply(create_date)
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
        
# Apply function
for yr in ['18', '19', '20']:
    movies[str(yr) + "00's"] = movies['Date'].apply(add_movie_year)

In [21]:
# and another
movies.head()

,movie_id,movie,genre,Date,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [22]:
def split_genres(val):
    try:
        if val.find(gene) >-1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

# Apply function for each genre
for gene in genres:        
    movies[gene] = movies['genre'].apply(split_genres)

In [23]:
# and a few more below, which you can use as necessary
movies.head()

,movie_id,movie,genre,Date,1800's,1900's,2000's,Adult,Short,Action,...,Thriller,Game-Show,War,News,Film-Noir,Sport,Crime,Animation,Biography,Romance
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
import datetime

change_timestamp = lambda x: datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [26]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-05 17:00:50
1,2,0102926,9,1590148016,2020-05-22 07:46:56
2,2,0208092,5,1586466072,2020-04-09 17:01:12
3,2,0358273,9,1579057827,2020-01-14 22:10:27
4,2,10039344,5,1578603053,2020-01-09 15:50:53


In [37]:
from datetime import datetime
reviews['month'] = reviews['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').month)

In [42]:
from datetime import datetime
reviews['day'] = reviews['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').day)

In [43]:
from datetime import datetime
reviews['year'] = reviews['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').year)

In [44]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month,day,year
0,1,0114508,8,1381006850,2013-10-05 17:00:50,2013,5,2013
1,2,0102926,9,1590148016,2020-05-22 07:46:56,2020,22,2020
2,2,0208092,5,1586466072,2020-04-09 17:01:12,2020,9,2020
3,2,0358273,9,1579057827,2020-01-14 22:10:27,2020,14,2020
4,2,10039344,5,1578603053,2020-01-09 15:50:53,2020,9,2020


In [46]:
reviews.to_csv('./reviews_clean.csv')
movies.to_csv('./movies_clean.csv')

In [47]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1    114508       8  1381006850  2013-10-05 17:00:50   
1           1        2    102926       9  1590148016  2020-05-22 07:46:56   
2           2        2    208092       5  1586466072  2020-04-09 17:01:12   
3           3        2    358273       9  1579057827  2020-01-14 22:10:27   
4           4        2  10039344       5  1578603053  2020-01-09 15:50:53   

   month  day  year  
0   2013    5  2013  
1   2020   22  2020  
2   2020    9  2020  
3   2020   14  2020  
4   2020    9  2020  
   Unnamed: 0  movie_id                                              movie  \
0           0         8      Edison Kinetoscopic Record of a Sneeze (1894)   
1           1        10                La sortie des usines Lumière (1895)   
2           2        12                      The Arrival of a Train (1896)   
3           3        25  The Oxford and Cambridge University Boat Race ...   
4           4  